In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import time
import os

In [2]:
file_path = "/data/projects/Tissue_specific_data/subsample/Data/all_input_stratified.feather"
#output_path = "/data/projects/Tissue_specific_data/Tissue_specific_transcript"

In [3]:
df= pd.read_feather(file_path)

In [4]:
df.head()

,sample_class,sample,ENST00000373020.8,ENST00000494424.1,ENST00000496771.5,ENST00000612152.4,ENST00000614008.4,ENST00000373031.4,ENST00000485971.1,ENST00000371582.8,...,ENST00000640157.1,ENST00000640623.1,ENST00000640630.1,ENST00000638486.1,ENST00000381568.9,ENST00000622217.1,ENST00000408734.1,ENST00000385204.1,ENST00000583027.1,ENST00000638236.1
0,Adipose_Tissue,GTEX-1117F-0226-SM-5GZZ7,26.32,0.0,2.53,0.47,0.23,8.73,2.84,0.91,...,0.0,0.0,0.0,0.0,0.00,0.48,0.0,0.0,0.0,0.0
1,Muscle,GTEX-1117F-0426-SM-5EGHI,3.95,0.0,0.48,0.21,0.24,0.44,0.15,1.73,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
2,Blood_Vessel,GTEX-1117F-0526-SM-5EGHJ,13.23,0.0,1.98,0.15,1.32,1.49,0.26,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
3,Blood_Vessel,GTEX-1117F-0626-SM-5N9CS,30.15,0.0,4.18,1.23,1.15,4.70,1.48,0.52,...,0.0,0.0,0.0,0.0,0.00,1.54,0.0,0.0,0.0,0.0
4,Heart,GTEX-1117F-0726-SM-5GIEN,6.60,0.0,0.38,0.31,0.74,0.03,0.00,0.00,...,0.1,0.0,0.0,0.0,0.07,0.68,0.0,0.0,0.0,0.0


In [5]:
df_p_values = pd.DataFrame()
df_p_values['Transcript_ID'] = df.columns[2:]

In [6]:
tissue_name = "Blood"
no_of_iteration = 3
# df_group1 = df[df['sample_class']==tissue_name]
# df_group2 = df[df['sample_class']!=tissue_name]

In [7]:
print(df.shape)
print(df_group1.shape)
print(df_group2.shape)

(17382, 163725)


NameError: name 'df_group1' is not defined

In [ ]:
df.groupby('sample_class').size()

In [114]:
def stratified_sample(df, iteration, tissue_of_interest,  strata, size_percentage=None, seed=None, keep_index= True):
    population = len(df)
    print(population, size_percentage)
    #size = __smpl_size(population, size)
    size = int(size_percentage * population)
    print(size)
    tmp = df.loc[:,strata]
    tmp['size'] = 1
    #print (tmp)
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    #print(tmp_grpd)
    tmp_grpd['samp_size'] = np.round(size/population * tmp_grpd['size']).astype(int)
    print(tmp_grpd)

    # controlling variable to create the dataframe or append to it
    first = True 
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        #print ("i", i)
        qry=''
        for s in range(len(strata)):
            stratum = strata[s]
            #print("##",s, stratum)
            value = tmp_grpd.iloc[i][stratum]
            #print(value)
            n = tmp_grpd.iloc[i]['samp_size']
            #print("N", n)

            if type(value) == str:
                value = "'" + str(value) + "'"
                #print(value)
            
            if s != len(strata)-1:
                qry = qry + stratum + ' == ' + str(value) +' & '
            else:
                qry = qry + stratum + ' == ' + str(value)
            #print('Query', qry)
        
        # final dataframe
        if first:
            stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            first = False
        else:
            tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            stratified_df = pd.concat([stratified_df,tmp_df], ignore_index=True)
    print(tissue_of_interest)
    cat1_df = stratified_df[stratified_df['sample_class']==tissue_of_interest]
    cat2_df = stratified_df[stratified_df['sample_class']!=tissue_of_interest]
    for index, row in df_p_values.iterrows():
        #print(row['Transcript_ID'])
        t, p = ttest_ind(cat1_df[row['Transcript_ID']], cat2_df[row['Transcript_ID']], equal_var=False)
        df_p_values.loc[index, ['ITER_{}'.format(iteration)]]= p

    
    return stratified_df

In [90]:
start_time  = time.time()
for i in range(no_of_iteration):
    stratified_sample(df=df, iteration =i, tissue_of_interest = tissue_name, strata=['sample_class'], size_percentage =0.7)
print(time.time()-start_time)

17382 0.8
13905
       sample_class  size  samp_size
0    Adipose_Tissue  1204        963
1     Adrenal_Gland   258        206
2           Bladder    21         17
3             Blood   929        743
4      Blood_Vessel  1335       1068
5             Brain  2642       2114
6            Breast   459        367
7      Cervix_Uteri    19         15
8             Colon   779        623
9         Esophagus  1445       1156
10   Fallopian_Tube     9          7
11            Heart   861        689
12           Kidney    89         71
13            Liver   226        181
14             Lung   578        462
15           Muscle   803        642
16            Nerve   619        495
17            Ovary   180        144
18         Pancreas   328        262
19        Pituitary   283        226
20         Prostate   245        196
21   Salivary_Gland   162        130
22             Skin  1809       1447
23  Small_Intestine   187        150
24           Spleen   241        193
25          Stomach   

In [91]:
df_p_values

,Transcript_ID,ITER_0,ITER_1,ITER_2
0,ENST00000373020.8,0.000000e+00,0.000000e+00,0.000000e+00
1,ENST00000494424.1,2.815337e-52,2.946357e-51,7.525728e-51
2,ENST00000496771.5,0.000000e+00,0.000000e+00,0.000000e+00
3,ENST00000612152.4,0.000000e+00,0.000000e+00,0.000000e+00
4,ENST00000614008.4,6.386024e-99,6.654822e-99,8.867693e-98
...,...,...,...,...
163718,ENST00000622217.1,3.423964e-11,2.541612e-11,5.301768e-12
163719,ENST00000408734.1,2.206672e-09,2.416088e-09,8.935349e-13
163720,ENST00000385204.1,3.632762e-04,5.348432e-04,1.799940e-03
163721,ENST00000583027.1,6.600304e-02,3.693325e-02,1.126524e-01


In [92]:
df_p_values.set_index('Transcript_ID', inplace=True)
print (df_p_values)

                         ITER_0        ITER_1        ITER_2
Transcript_ID                                              
ENST00000373020.8  0.000000e+00  0.000000e+00  0.000000e+00
ENST00000494424.1  2.815337e-52  2.946357e-51  7.525728e-51
ENST00000496771.5  0.000000e+00  0.000000e+00  0.000000e+00
ENST00000612152.4  0.000000e+00  0.000000e+00  0.000000e+00
ENST00000614008.4  6.386024e-99  6.654822e-99  8.867693e-98
...                         ...           ...           ...
ENST00000622217.1  3.423964e-11  2.541612e-11  5.301768e-12
ENST00000408734.1  2.206672e-09  2.416088e-09  8.935349e-13
ENST00000385204.1  3.632762e-04  5.348432e-04  1.799940e-03
ENST00000583027.1  6.600304e-02  3.693325e-02  1.126524e-01
ENST00000638236.1  0.000000e+00  0.000000e+00  0.000000e+00

[163723 rows x 3 columns]


In [103]:
def count(df):
    #n=df.shape[1]
    #print(df)
    threshold_values=[0.05]
    for threshold in threshold_values:
        r = np.sum(df >= threshold)
        #print (r, df, no_of_iteration)
        df['>={}'.format(threshold)] = (r+1) /(no_of_iteration+1)
    #df['r'] = np.sum(df == 'r')
    return df

In [104]:
start_time = time.time()
temp_df_empirical = df_p_values.apply(count, axis = 1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 66.26727819442749 seconds ---


In [105]:
temp_df_empirical

,ITER_0,ITER_1,ITER_2,>=0.05
Transcript_ID,,,,
ENST00000373020.8,0.000000e+00,0.000000e+00,0.000000e+00,0.25
ENST00000494424.1,2.815337e-52,2.946357e-51,7.525728e-51,0.25
ENST00000496771.5,0.000000e+00,0.000000e+00,0.000000e+00,0.25
ENST00000612152.4,0.000000e+00,0.000000e+00,0.000000e+00,0.25
ENST00000614008.4,6.386024e-99,6.654822e-99,8.867693e-98,0.25
...,...,...,...,...
ENST00000622217.1,3.423964e-11,2.541612e-11,5.301768e-12,0.25
ENST00000408734.1,2.206672e-09,2.416088e-09,8.935349e-13,0.25
ENST00000385204.1,3.632762e-04,5.348432e-04,1.799940e-03,0.25


In [34]:
df_mean = df.groupby(tissue_name).mean(numeric_only=True)
index_list = list(df_mean.index.values)

,index,sample_class,sample,ENST00000373020.8,ENST00000494424.1,ENST00000496771.5,ENST00000612152.4,ENST00000614008.4,ENST00000373031.4,ENST00000485971.1,...,ENST00000640157.1,ENST00000640623.1,ENST00000640630.1,ENST00000638486.1,ENST00000381568.9,ENST00000622217.1,ENST00000408734.1,ENST00000385204.1,ENST00000583027.1,ENST00000638236.1
0,10615,Adipose_Tissue,GTEX-1K9T9-0226-SM-E9U6K,23.12,0.0,0.52,0.98,0.61,44.83,7.91,...,0.0,0.00,0.0,0.1,0.00,0.00,0.0,0.0,0.0,0.00
1,8241,Adipose_Tissue,GTEX-1GMRU-0226-SM-9YFLW,27.44,0.0,2.31,1.29,0.97,46.13,12.50,...,0.0,0.00,0.0,0.0,0.00,0.12,0.0,0.0,0.0,0.15
2,9442,Adipose_Tissue,GTEX-1I1CD-1026-SM-COH3G,36.05,0.0,3.80,1.29,0.50,4.45,2.68,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.05
3,11009,Adipose_Tissue,GTEX-1LSNL-0526-SM-DIPFI,18.77,0.0,0.96,1.09,0.56,17.69,2.36,...,0.0,0.00,0.0,0.0,0.00,1.14,0.0,0.0,0.0,0.09
4,15630,Adipose_Tissue,GTEX-XV7Q-1726-SM-4BRUU,23.23,0.0,2.41,1.39,0.07,1.20,0.29,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13156,17169,Vagina,GTEX-ZXES-0626-SM-5E45P,26.12,0.0,1.15,0.77,0.65,0.06,0.00,...,0.0,0.02,0.0,0.0,0.00,0.49,0.0,0.0,0.0,0.40
13157,6377,Vagina,GTEX-18A7A-2826-SM-7LT8J,25.81,0.0,1.41,0.63,0.00,0.25,0.21,...,0.0,0.00,0.0,0.0,0.00,1.80,0.0,0.0,0.0,0.00
13158,2603,Vagina,GTEX-13N11-1526-SM-5L3FL,24.88,0.0,8.22,2.97,2.48,0.11,0.28,...,0.0,0.00,0.0,0.0,0.02,0.36,0.0,0.0,0.0,1.38
13159,258,Vagina,GTEX-11DXX-1726-SM-5H11T,24.70,0.0,1.95,0.76,1.83,0.24,0.00,...,0.0,0.00,0.0,0.0,0.29,0.00,0.0,0.0,0.0,0.25


In [37]:
temp_df.groupby(['sample_class']).size()

sample_class
Adipose_Tissue      963
Adrenal_Gland       206
Bladder              17
Blood_Vessel       1068
Brain              2114
Breast              367
Cervix_Uteri         15
Colon               623
Esophagus          1156
Fallopian_Tube        7
Heart               689
Kidney               71
Liver               181
Lung                462
Muscle              642
Nerve               495
Ovary               144
Pancreas            262
Pituitary           226
Prostate            196
Salivary_Gland      130
Skin               1447
Small_Intestine     150
Spleen              193
Stomach             287
Testis              289
Thyroid             522
Uterus              114
Vagina              125
dtype: int64